In [1]:
# 第一种方法，利用反射机制推断RDD模式
from pyspark.sql.types import Row

In [2]:
def f(x):
    rel = {}
    rel['name'] = x[0]
    rel['age'] = x[1]
    return rel

In [3]:
from pyspark import SparkContext

In [4]:
sc = SparkContext('local','test')

In [8]:
from pyspark.sql import SparkSession

In [9]:
SparkSession(sc)

In [10]:
peopleDF = sc.textFile("../../../dataset/people.txt").map(lambda line : line.split(',')).map(lambda x: Row(**f(x))).toDF()

In [11]:
peopleDF.createOrReplaceTempView('people') #必须注册为临时表才能供下面的查询使用

In [13]:
from pyspark.conf import SparkConf
from pyspark.sql.session import SparkSession

In [14]:
spark = SparkSession.builder.config(conf=SparkConf()).getOrCreate()

In [15]:
personsDF = spark.sql("select * from people")

In [31]:
# foreah的参数是一个function，触发分区所在的机器执行相应操作，分布式情况下数据不在本地driver程序，数据不在jupyter进程中，collect一下，数据就会读取到jupyter

In [19]:
personsDF.rdd.map(lambda t:"Name:"+t[0]+","+"Age:"+t[1]).foreach(print)

In [21]:
# 第二种方法，使用createDataFrame(rdd,schema)编码方式定义RDD模式
from pyspark.sql.types import Row
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType

In [22]:
peopleRDD = sc.textFile("../../../dataset/people.txt")

In [23]:
# 定义一个模式字符串
schemaString = "name age"
# 根据模式字符串生成模式
fields = list(map(lambda fieldName : StructField(fieldName,StringType(),nullable=True),schemaString.split(" ")))
schema = StructType(fields) # 从上面信息可以看出，schema描述了模式信息，模式中包含name和age两个字段

In [25]:
rowRDD = peopleRDD.map(lambda line:line.split(',')).map(lambda attributes:Row(attributes[0],attributes[1]))

In [26]:
peopleDF = spark.createDataFrame(rowRDD,schema)

In [28]:
peopleDF.createOrReplaceTempView("people")

In [29]:
results = spark.sql("SELECT * FROM people")

In [30]:
results.rdd.map( lambda attributes : "name: " + attributes[0]+","+"age:"+attributes[1]).collect()

['name: Michael,age: 29', 'name: Andy,age: 30', 'name: Justin,age: 19']